In [ ]:
# =============================================================================
# ⚙️ CONFIGURATION CENTRALISÉE
# =============================================================================

import sys
import os
import time
from pathlib import Path

# =============================================================================
# 🌍 DÉTECTION AUTOMATIQUE DE L'ENVIRONNEMENT
# =============================================================================

def detect_environment():
    """Détecte automatiquement si on est sur Colab ou en local"""
    try:
        import google.colab
        return True
    except ImportError:
        return False

# =============================================================================
# 📋 CONFIGURATION PRINCIPALE
# =============================================================================

class Config:
    """Configuration centralisée du projet"""
    
    # 🌍 Environnement
    USING_COLAB = detect_environment()
    
    # 🎬 VIDÉO ET PROJET  
    VIDEO_NAME = "SD_13_06_2025_cam1_PdB_S1_T959s_1"  # ⚠️ MODIFIEZ ICI
    FRAME_INTERVAL = 3                            # ⚠️ MODIFIEZ ICI
    
    # 🎬 OPTIONS D'EXTRACTION
    EXTRACT_FRAMES = True                         # ⚠️ MODIFIEZ ICI
    FORCE_EXTRACTION = True                      # ⚠️ MODIFIEZ ICI
    
    # 🎯 SEGMENTATION VIDÉO (NOUVEAU)
    SEGMENT_MODE = True                          # ⚠️ MODIFIEZ ICI - Active le mode segmentation
    
    # 🕐 OFFSETS EN SECONDES (RECOMMANDÉ)
    SEGMENT_OFFSET_BEFORE_SECONDS = 2.0         # ⚠️ MODIFIEZ ICI - Secondes AVANT la frame de référence
    SEGMENT_OFFSET_AFTER_SECONDS = 2.0          # ⚠️ MODIFIEZ ICI - Secondes APRÈS la frame de référence
    
    # 🎬 OFFSETS EN FRAMES (OPTIONNEL - sera calculé automatiquement si secondes définies)
    SEGMENT_OFFSET_BEFORE = None                 # ⚠️ MODIFIEZ ICI - Frames AVANT (ou None pour auto-calcul)
    SEGMENT_OFFSET_AFTER = None                  # ⚠️ MODIFIEZ ICI - Frames APRÈS (ou None pour auto-calcul)
    
    # 🤖 SAM2 CONFIGURATION
    SAM2_MODEL = "sam2.1_hiera_l"                # ou "sam2.1_hiera_s" pour small
    SAM2_CHECKPOINT = "sam2.1_hiera_large.pt"    # ou "sam2.1_hiera_small.pt"
    
    # 🗂️ CHEMINS AUTOMATIQUES
    @property
    def videos_dir(self):
        return Path("./videos") if self.USING_COLAB else Path("../data/videos")
    
    @property 
    def checkpoint_path(self):
        base = "../../checkpoints" if self.USING_COLAB else "../checkpoints"
        return Path(base) / self.SAM2_CHECKPOINT
    
    @property
    def model_config_path(self):
        return f"configs/sam2.1/{self.SAM2_MODEL}.yaml"
    
    # 🎬 CHEMINS VIDÉO
    @property
    def video_path(self):
        return self.videos_dir / f"{self.VIDEO_NAME}.mp4"
    
    @property
    def config_path(self):
        return self.videos_dir / f"{self.VIDEO_NAME}_config.json"
    
    @property
    def output_dir(self):
        return self.videos_dir / "outputs" / self.VIDEO_NAME
    
    @property
    def frames_dir(self):
        return self.output_dir / "frames"
    
    @property
    def masks_dir(self):
        return self.output_dir / "masks"
    
    @property
    def output_video_path(self):
        return self.output_dir / f"{self.VIDEO_NAME}_annotated.mp4"
    
    @property
    def output_json_path(self):
        return self.output_dir / f"{self.VIDEO_NAME}_project.json"
    
    def setup_directories(self):
        """Crée tous les dossiers nécessaires (sans vérifier les fichiers)"""
        print("🏗️ Création de la structure de dossiers...")
        self.videos_dir.mkdir(parents=True, exist_ok=True)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.frames_dir.mkdir(exist_ok=True)
        self.masks_dir.mkdir(exist_ok=True)
        print(f"   ✅ Dossiers créés dans: {self.output_dir}")
    
    def check_files_exist(self):
        """Vérifie si les fichiers requis existent (sans lever d'erreur)"""
        video_exists = self.video_path.exists()
        config_exists = self.config_path.exists()
        
        print(f"📄 Vérification des fichiers:")
        print(f"   🎬 Vidéo: {'✅' if video_exists else '❌'} {self.video_path}")
        print(f"   📄 Config: {'✅' if config_exists else '❌'} {self.config_path}")
        
        return video_exists and config_exists
    
    def wait_for_files(self, max_wait_minutes=10, check_interval=10):
        """Attend que les fichiers soient disponibles (utile pour Colab)"""
        if not self.USING_COLAB:
            # En local, validation immédiate
            if not self.check_files_exist():
                missing = []
                if not self.video_path.exists():
                    missing.append(f"vidéo: {self.video_path}")
                if not self.config_path.exists():
                    missing.append(f"config: {self.config_path}")
                raise FileNotFoundError(f"❌ Fichiers manquants: {', '.join(missing)}")
            return True
        
        # Sur Colab, attente avec timeout
        print(f"⏳ Attente des fichiers (max {max_wait_minutes}min)...")
        start_time = time.time()
        max_wait_seconds = max_wait_minutes * 60
        
        while time.time() - start_time < max_wait_seconds:
            if self.check_files_exist():
                print("✅ Tous les fichiers sont disponibles!")
                return True
            
            elapsed = int(time.time() - start_time)
            remaining = max_wait_seconds - elapsed
            print(f"   ⏳ Attente... ({elapsed}s écoulées, {remaining}s restantes)")
            time.sleep(check_interval)
        
        print(f"⚠️ Timeout atteint ({max_wait_minutes}min)")
        return False
    
    def validate_files_now(self):
        """Validation immédiate avec erreur si fichiers manquants"""
        if not self.video_path.exists():
            raise FileNotFoundError(f"❌ Vidéo non trouvée: {self.video_path}")
        if not self.config_path.exists():
            raise FileNotFoundError(f"❌ Fichier config non trouvé: {self.config_path}")
        print("✅ Tous les fichiers sont valides")
    
    def get_video_fps(self):
        """Récupère le FPS de la vidéo"""
        if not self.video_path.exists():
            print(f"⚠️  Vidéo non trouvée pour récupérer le FPS: {self.video_path}")
            return 25.0  # FPS par défaut
        
        import cv2
        cap = cv2.VideoCapture(str(self.video_path))
        if not cap.isOpened():
            print(f"⚠️  Impossible d'ouvrir la vidéo pour récupérer le FPS")
            return 25.0  # FPS par défaut
        
        fps = cap.get(cv2.CAP_PROP_FPS)
        cap.release()
        return fps if fps > 0 else 25.0  # FPS par défaut si invalide
    
    def seconds_to_frames(self, seconds: float, fps: float = None) -> int:
        """Convertit des secondes en nombre de frames ENTIÈRES"""
        if fps is None:
            fps = self.get_video_fps()
        # ✅ CORRECTION: s'assurer que le résultat est un entier (frame entière)
        return int(round(seconds * fps))
    
    def frames_to_seconds(self, frames: int, fps: float = None) -> float:
        """Convertit des frames en secondes"""
        if fps is None:
            fps = self.get_video_fps()
        return frames / fps
    
    def get_segment_offsets_frames(self):
        """Calcule les offsets en frames ENTIÈRES, priorité aux secondes si définies"""
        fps = self.get_video_fps()
        
        # Priorité aux offsets en secondes
        if hasattr(self, 'SEGMENT_OFFSET_BEFORE_SECONDS') and self.SEGMENT_OFFSET_BEFORE_SECONDS is not None:
            offset_before_frames = self.seconds_to_frames(self.SEGMENT_OFFSET_BEFORE_SECONDS, fps)
        elif hasattr(self, 'SEGMENT_OFFSET_BEFORE') and self.SEGMENT_OFFSET_BEFORE is not None:
            # ✅ CORRECTION: s'assurer que même les frames définies manuellement sont des entiers
            offset_before_frames = int(self.SEGMENT_OFFSET_BEFORE)
        else:
            offset_before_frames = self.seconds_to_frames(2.0, fps)  # Défaut: 2 secondes
        
        if hasattr(self, 'SEGMENT_OFFSET_AFTER_SECONDS') and self.SEGMENT_OFFSET_AFTER_SECONDS is not None:
            offset_after_frames = self.seconds_to_frames(self.SEGMENT_OFFSET_AFTER_SECONDS, fps)
        elif hasattr(self, 'SEGMENT_OFFSET_AFTER') and self.SEGMENT_OFFSET_AFTER is not None:
            # ✅ CORRECTION: s'assurer que même les frames définies manuellement sont des entiers
            offset_after_frames = int(self.SEGMENT_OFFSET_AFTER)
        else:
            offset_after_frames = self.seconds_to_frames(2.0, fps)  # Défaut: 2 secondes
        
        return offset_before_frames, offset_after_frames
    
    def display_config(self):
        """Affiche la configuration actuelle"""
        print(f"📋 CONFIGURATION CENTRALISÉE:")
        print(f"   🌍 Environnement: {'🔬 Colab' if self.USING_COLAB else '🖥️ Local'}")
        print(f"   🎬 Vidéo: {self.VIDEO_NAME}")
        print(f"   ⏯️  Intervalle frames: {self.FRAME_INTERVAL}")
        print(f"   🎬 Extraction: {'✅ Activée' if self.EXTRACT_FRAMES else '❌ Désactivée'}")
        print(f"   🔄 Force extraction: {'✅ Oui' if self.FORCE_EXTRACTION else '❌ Non'}")
        print(f"   🎯 Segmentation: {'✅ Activée' if getattr(self, 'SEGMENT_MODE', False) else '❌ Désactivée'}")
        
        if getattr(self, 'SEGMENT_MODE', False):
            # Récupérer les informations vidéo
            fps = self.get_video_fps() if self.video_path.exists() else 25.0
            offset_before_frames, offset_after_frames = self.get_segment_offsets_frames()
            
            # Afficher les offsets en secondes et frames
            offset_before_seconds = self.frames_to_seconds(offset_before_frames, fps)
            offset_after_seconds = self.frames_to_seconds(offset_after_frames, fps)
            
            print(f"   🎥 FPS vidéo: {fps:.1f}")
            print(f"   🕐 Offset avant: {offset_before_seconds:.1f}s ({offset_before_frames} frames)")
            print(f"   🕐 Offset après: {offset_after_seconds:.1f}s ({offset_after_frames} frames)")
            
        print(f"   🤖 Modèle SAM2: {self.SAM2_MODEL}")
        print(f"   📁 Dossier vidéos: {self.videos_dir}")
        print(f"   📄 Fichier config: {self.config_path}")
        print(f"   📁 Sortie: {self.output_dir}")
        print(f"   💾 Checkpoint: {self.checkpoint_path}")

# =============================================================================
# 🚀 INITIALISATION
# =============================================================================

# Création de l'instance de configuration
cfg = Config()

# Setup automatique des dossiers (toujours safe)
cfg.setup_directories()
cfg.display_config()

# Vérification des fichiers selon l'environnement
if cfg.USING_COLAB:
    print(f"\n🔬 Mode Colab détecté:")
    print(f"   💡 Les dossiers sont créés, vous pouvez maintenant uploader vos fichiers")
    print(f"   📤 Uploadez dans: {cfg.videos_dir}")
    print(f"   📄 Fichiers attendus:")
    print(f"      • {cfg.video_path.name}")
    print(f"      • {cfg.config_path.name}")
    
    # Vérification simple sans erreur
    cfg.check_files_exist()
    print(f"   💡 Utilisez cfg.wait_for_files() quand les uploads sont terminés")
else:
    print(f"\n🖥️ Mode Local détecté:")
    # Validation immédiate en local
    cfg.validate_files_now()

print("\n✅ Configuration centralisée initialisée!")

🏗️ Création de la structure de dossiers...
   ✅ Dossiers créés dans: ..\data\videos\outputs\SD_13_06_2025_cam1_PdB_S1_T959s_1
📋 CONFIGURATION CENTRALISÉE:
   🌍 Environnement: 🖥️ Local
   🎬 Vidéo: SD_13_06_2025_cam1_PdB_S1_T959s_1
   ⏯️  Intervalle frames: 3
   🎬 Extraction: ✅ Activée
   🔄 Force extraction: ✅ Oui
   🎯 Segmentation: ✅ Activée
   🎥 FPS vidéo: 25.0
   🕐 Offset avant: 2.0s (50 frames)
   🕐 Offset après: 2.0s (50 frames)
   🤖 Modèle SAM2: sam2.1_hiera_l
   📁 Dossier vidéos: ..\data\videos
   📄 Fichier config: ..\data\videos\SD_13_06_2025_cam1_PdB_S1_T959s_1_config.json
   📁 Sortie: ..\data\videos\outputs\SD_13_06_2025_cam1_PdB_S1_T959s_1
   💾 Checkpoint: ..\checkpoints\sam2.1_hiera_large.pt

🖥️ Mode Local détecté:
✅ Tous les fichiers sont valides

✅ Configuration centralisée initialisée!


In [2]:
# =============================================================================
# 🔧 INSTALLATION ET SETUP SAM2 (AUTO-INSTALL COLAB)
# =============================================================================

def is_sam2_installed():
    """Vérifie si SAM 2 est déjà installé"""
    try:
        import sam2
        return True
    except ImportError:
        return False

def install_sam2_colab(cfg):
    """Installation complète de SAM2 sur Colab avec chemins de la config"""
    print("🔧 Installation de SAM 2 en cours...")
    
    # Packages de base
    print("   📦 Installation des dépendances...")
    !{sys.executable} -m pip install opencv-python matplotlib
    !{sys.executable} -m pip install 'git+https://github.com/facebookresearch/sam2.git'
    
    # Création du dossier checkpoints (utilise la config)
    checkpoint_dir = cfg.checkpoint_path.parent
    print(f"   📁 Création du dossier checkpoints: {checkpoint_dir}")
    checkpoint_dir.mkdir(parents=True, exist_ok=True)
    
    # Téléchargement du modèle configuré
    checkpoint_url = "https://dl.fbaipublicfiles.com/segment_anything_2/092824"
    model_file = cfg.SAM2_CHECKPOINT
    
    print(f"   ⬇️ Téléchargement du modèle: {model_file}")
    !wget -P {checkpoint_dir} -q {checkpoint_url}/{model_file}
    
    # Optionnel: télécharger l'autre modèle si besoin
    if "large" in model_file:
        other_model = "sam2.1_hiera_small.pt"
        print(f"   📦 Modèle small également disponible: {other_model}")
        # !wget -P {checkpoint_dir} -q {checkpoint_url}/{other_model}
    else:
        other_model = "sam2.1_hiera_large.pt"
        print(f"   📦 Modèle large également disponible: {other_model}")
        # !wget -P {checkpoint_dir} -q {checkpoint_url}/{other_model}
    
    # Nettoyage Colab
    if cfg.USING_COLAB:
        print("   🧹 Nettoyage des fichiers temporaires Colab...")
        !rm -rf /content/sample_data/* 2>/dev/null || true
    
    # Nettoyage mémoire
    import gc
    import torch
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        print(f"   🔥 Mémoire GPU nettoyée")
    
    print("✅ Installation de SAM 2 terminée")

# =============================================================================
# 🚀 AUTO-SETUP SAM2 INTELLIGENT
# =============================================================================

print("🤖 Vérification et setup SAM2...")

# Vérifications
sam2_installed = is_sam2_installed()
checkpoint_available = cfg.checkpoint_path.exists()

print(f"   📦 SAM2 installé: {'✅' if sam2_installed else '❌'}")
print(f"   💾 Checkpoint disponible: {'✅' if checkpoint_available else '❌'} {cfg.checkpoint_path}")

# Logique d'installation
if cfg.USING_COLAB and (not sam2_installed or not checkpoint_available):
    print(f"🔬 Colab détecté - Installation automatique...")
    install_sam2_colab(cfg)
    
elif cfg.USING_COLAB and sam2_installed and checkpoint_available:
    print("✅ SAM 2 déjà installé sur Colab - SKIP installation")
    
elif not cfg.USING_COLAB:
    print("🖥️ Mode local détecté")
    if not sam2_installed:
        print("   ⚠️ SAM2 non installé. Installez avec: pip install git+https://github.com/facebookresearch/sam2.git")
    if not checkpoint_available:
        print(f"   ⚠️ Checkpoint manquant: {cfg.checkpoint_path}")
        print("   💡 Téléchargez depuis: https://github.com/facebookresearch/sam2#download-checkpoints")
    
    if sam2_installed and checkpoint_available:
        print("✅ SAM2 prêt en mode local")

print("✅ Setup SAM2 terminé")


🤖 Vérification et setup SAM2...
   📦 SAM2 installé: ✅
   💾 Checkpoint disponible: ✅ ..\checkpoints\sam2.1_hiera_large.pt
🖥️ Mode local détecté
✅ SAM2 prêt en mode local
✅ Setup SAM2 terminé


In [3]:
# =============================================================================
# 📦 IMPORTS ET CONFIGURATION ENVIRONNEMENT
# =============================================================================

# ==================== IMPORTS SYSTÈME ====================
import sys
import os
import json
import time
from pathlib import Path
from datetime import datetime
from typing import List, Tuple, Dict, Any, Optional
from dataclasses import dataclass
import uuid
import base64

# ==================== IMPORTS SCIENTIFIQUES ====================
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt

# ==================== IMPORTS DEEP LEARNING ====================
import torch
import torchvision
from pycocotools.mask import encode as encode_rle

# =============================================================================
# 🖥️ CONFIGURATION AUTOMATIQUE DU DEVICE ET OPTIMISATIONS
# =============================================================================

def setup_torch_environment(verbose=True):
    """Configure automatiquement l'environnement PyTorch optimal"""
    
    if verbose:
        print("🔧 Configuration de l'environnement PyTorch...")
        print(f"   📦 PyTorch version: {torch.__version__}")
        print(f"   📦 Torchvision version: {torchvision.__version__}")
    
    # === SÉLECTION DU DEVICE ===
    if torch.cuda.is_available():
        device = torch.device("cuda")
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        if verbose:
            print(f"   🔥 CUDA disponible: {gpu_name} ({gpu_memory:.1f}GB)")
    elif torch.backends.mps.is_available():
        device = torch.device("mps")
        if verbose:
            print(f"   🍎 MPS (Apple Silicon) disponible")
    else:
        device = torch.device("cpu")
        if verbose:
            print(f"   💻 CPU seulement")
    
    # === OPTIMISATIONS CUDA ===
    optimizations_applied = []
    
    if device.type == "cuda":
        # Autocast pour économiser la mémoire
        torch.autocast("cuda", dtype=torch.bfloat16).__enter__()
        optimizations_applied.append("Autocast bfloat16")
        
        # Optimisations TensorFloat-32 (si GPU récent)
        gpu_compute_capability = torch.cuda.get_device_properties(0).major
        if gpu_compute_capability >= 8:  # Ampere et plus récent
            torch.backends.cuda.matmul.allow_tf32 = True
            torch.backends.cudnn.allow_tf32 = True
            optimizations_applied.append("TensorFloat-32")
        
        # Optimisations mémoire additionnelles
        torch.backends.cudnn.benchmark = True  # Optimise pour tailles fixes
        optimizations_applied.append("cuDNN benchmark")
    
    if verbose and optimizations_applied:
        print(f"   ⚡ Optimisations activées: {', '.join(optimizations_applied)}")
    
    return device, optimizations_applied

def display_system_info(device):
    """Affiche les informations système détaillées"""
    print(f"\n📊 INFORMATIONS SYSTÈME:")
    print(f"   🖥️  Device principal: {device}")
    
    if device.type == "cuda":
        print(f"   🔥 GPU: {torch.cuda.get_device_name(0)}")
        print(f"   💾 Mémoire GPU: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f}GB")
        print(f"   🧮 Compute Capability: {torch.cuda.get_device_properties(0).major}.{torch.cuda.get_device_properties(0).minor}")
    
    # CPU Info
    import platform
    print(f"   💻 CPU: {platform.processor()}")
    print(f"   🧠 Threads: {torch.get_num_threads()}")
    
    # Versions importantes
    print(f"   🐍 Python: {platform.python_version()}")
    print(f"   📦 NumPy: {np.__version__}")
    print(f"   🎥 OpenCV: {cv2.__version__}")

# =============================================================================
# 🚀 INITIALISATION ENVIRONNEMENT
# =============================================================================

# Configuration automatique
device, optimizations = setup_torch_environment(verbose=True)

# Ajout du device à notre configuration centralisée
if 'cfg' in globals():
    cfg.device = device
    cfg.torch_optimizations = optimizations
    print(f"✅ Device ajouté à la configuration: cfg.device = {device}")
else:
    print(f"⚠️ Configuration cfg non trouvée - device disponible en tant que variable globale")

# Affichage des informations système (optionnel, décommentez si besoin)
# display_system_info(device)

# Nettoyage initial de la mémoire
if device.type == "cuda":
    torch.cuda.empty_cache()

print(f"\n✅ Environnement PyTorch configuré et optimisé!")

🔧 Configuration de l'environnement PyTorch...
   📦 PyTorch version: 2.5.1+cu121
   📦 Torchvision version: 0.20.1+cu121
   🔥 CUDA disponible: NVIDIA GeForce GTX 1650 with Max-Q Design (4.0GB)
   ⚡ Optimisations activées: Autocast bfloat16, cuDNN benchmark
✅ Device ajouté à la configuration: cfg.device = cuda

✅ Environnement PyTorch configuré et optimisé!


In [4]:
# =============================================================================
# 📄 CHARGEMENT ET VALIDATION DE LA CONFIGURATION JSON
# =============================================================================

def load_and_validate_project_config(config_path: Path) -> Dict[str, Any]:
    """Charge et valide le fichier de configuration JSON du projet."""
    print(f"📄 Chargement de la configuration projet: {config_path}")

    # Vérification existence du fichier
    if not config_path.exists():
        raise FileNotFoundError(f"❌ Fichier config non trouvé: {config_path}")

    try:
        with open(config_path, 'r', encoding='utf-8') as f:
            config = json.load(f)
    except json.JSONDecodeError as e:
        raise ValueError(f"❌ Erreur JSON dans {config_path}: {e}")

    # === VALIDATION DE LA STRUCTURE ===
    required_sections = ['calibration', 'objects', 'initial_annotations']
    missing_sections = [section for section in required_sections if section not in config]
    
    if missing_sections:
        raise ValueError(f"❌ Sections manquantes dans le config: {missing_sections}")

    # === VALIDATION DES DONNÉES ===
    
    # Validation calibration
    if 'camera_parameters' not in config['calibration']:
        raise ValueError("❌ 'camera_parameters' manquant dans la calibration")
    
    # Validation objets
    if not config['objects']:
        raise ValueError("❌ Aucun objet défini dans la configuration")
    
    # Validation annotations initiales
    if not config['initial_annotations']:
        raise ValueError("❌ Aucune annotation initiale définie")

    # === STATISTIQUES ET RÉSUMÉ ===
    num_objects = len(config['objects'])
    
    # Comptage des annotations
    total_annotations = 0
    annotation_frames = set()
    for frame_data in config['initial_annotations']:
        frame_idx = frame_data.get('frame')
        annotations = frame_data.get('annotations', [])
        total_annotations += len(annotations)
        annotation_frames.add(frame_idx)

    # Types d'objets
    obj_types = {}
    obj_by_type = {}
    for obj in config['objects']:
        obj_type = obj.get('obj_type', 'unknown')
        obj_types[obj_type] = obj_types.get(obj_type, 0) + 1
        if obj_type not in obj_by_type:
            obj_by_type[obj_type] = []
        obj_by_type[obj_type].append(obj.get('obj_id', 'no_id'))

    print(f"✅ Configuration projet chargée et validée:")
    print(f"   📷 Calibration caméra: ✅ OK")
    print(f"   🎯 Objets définis: {num_objects}")
    print(f"   📍 Annotations initiales: {total_annotations} sur {len(annotation_frames)} frames")
    print(f"   🏷️  Types d'objets:")
    for obj_type, count in obj_types.items():
        ids = obj_by_type[obj_type]
        print(f"      • {obj_type}: {count} ({ids})")

    return config

# =============================================================================
# 🚀 CHARGEMENT DE LA CONFIGURATION
# =============================================================================

# Chargement avec gestion intelligente Colab/Local
if cfg.USING_COLAB:
    print("🔬 Mode Colab détecté pour le chargement config")
    if cfg.config_path.exists():
        project_config = load_and_validate_project_config(cfg.config_path)
    else:
        print(f"⏳ Fichier config pas encore uploadé, utilisez:")
        print(f"   project_config = wait_and_load_config(cfg)")
        project_config = None
else:
    print("🖥️ Mode local: chargement immédiat")
    project_config = load_and_validate_project_config(cfg.config_path)

if project_config:
    print("\n✅ Configuration projet prête à l'usage!")

🖥️ Mode local: chargement immédiat
📄 Chargement de la configuration projet: ..\data\videos\SD_13_06_2025_cam1_PdB_S1_T959s_1_config.json
✅ Configuration projet chargée et validée:
   📷 Calibration caméra: ✅ OK
   🎯 Objets définis: 13
   📍 Annotations initiales: 13 sur 1 frames
   🏷️  Types d'objets:
      • player: 12 ([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12])
      • ball: 1 ([13])

✅ Configuration projet prête à l'usage!


In [5]:
# =============================================================================
# 🎯 FONCTIONS DE SEGMENTATION VIDÉO
# =============================================================================

def calculate_segment_bounds(reference_frame: int, offset_before: int, offset_after: int,
                           total_frames: int, frame_interval: int = 1) -> tuple:
    """
    Calcule les bornes du segment vidéo à traiter.
    
    Args:
        reference_frame: Frame de référence (celle avec l'annotation initiale)
        offset_before: Nombre de frames à prendre AVANT la frame de référence
        offset_after: Nombre de frames à prendre APRÈS la frame de référence
        total_frames: Nombre total de frames dans la vidéo
        frame_interval: Intervalle entre les frames traitées
    
    Returns:
        (start_frame, end_frame, processed_start_idx, processed_end_idx)
    """
    
    # Calcul des bornes en frames originales
    start_frame = max(0, reference_frame - offset_before)
    end_frame = min(total_frames - 1, reference_frame + offset_after)
    
    # Conversion en indices de frames traitées
    processed_start_idx = start_frame // frame_interval
    processed_end_idx = end_frame // frame_interval
    
    print(f"🎯 CALCUL DES BORNES DE SEGMENTATION:")
    print(f"   📍 Frame de référence: {reference_frame}")
    print(f"   📉 Offset avant: {offset_before} frames")
    print(f"   📈 Offset après: {offset_after} frames")
    print(f"   🎬 Segment original: frames {start_frame} à {end_frame}")
    print(f"   🎬 Segment traité: indices {processed_start_idx} à {processed_end_idx}")
    print(f"   📊 Nombre de frames à traiter: {processed_end_idx - processed_start_idx + 1}")
    
    return start_frame, end_frame, processed_start_idx, processed_end_idx

def extract_segment_frames(video_path, frames_dir, start_frame: int, end_frame: int,
                         frame_interval: int = 1, force_extraction: bool = False) -> int:
    """
    Extrait seulement les frames du segment spécifié avec nommage séquentiel.
    
    Args:
        video_path: Chemin vers la vidéo
        frames_dir: Dossier de destination
        start_frame: Frame de début
        end_frame: Frame de fin
        frame_interval: Intervalle entre frames
        force_extraction: Force la ré-extraction
    
    Returns:
        Nombre de frames extraites
    """
    
    print(f"🎬 EXTRACTION DU SEGMENT:")
    print(f"   📹 Vidéo: {video_path}")
    print(f"   📁 Destination: {frames_dir}")
    print(f"   🎯 Segment: frames {start_frame} à {end_frame}")
    print(f"   ⏯️  Intervalle: {frame_interval}")    
    
    # ✅ CORRECTION: Calculer les frames attendues avec nommage séquentiel
    expected_frames = []
    sequential_idx = 0  # ← Compteur séquentiel pour le nommage
    
    for frame_idx in range(start_frame, end_frame + 1, frame_interval):
        expected_frames.append((frame_idx, sequential_idx))  # (frame_originale, index_séquentiel)
        sequential_idx += 1
    
    # Vérifier si extraction déjà faite avec le nouveau nommage
    existing_frames = []
    for frame_idx, seq_idx in expected_frames:
        filename = frames_dir / f"{seq_idx:05d}.jpg"  # ← Utiliser l'index séquentiel
        if filename.exists():
            existing_frames.append(filename)
    
    if len(existing_frames) == len(expected_frames) and not force_extraction:
        print(f"📂 {len(existing_frames)} frames du segment déjà extraites - SKIP")
        return len(existing_frames)
    elif existing_frames and force_extraction:
        print(f"🔄 {len(existing_frames)} frames existantes - SUPPRESSION et ré-extraction...")
        for frame_file in existing_frames:
            frame_file.unlink()
        print(f"🗑️  Frames existantes supprimées")
    
    # Extraction
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise ValueError(f"❌ Impossible d'ouvrir la vidéo: {video_path}")
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    print(f"📊 Vidéo: {total_frames} frames total, {fps:.1f} FPS")
    print(f"📊 Nommage: séquentiel de 00000.jpg à {len(expected_frames)-1:05d}.jpg")
    
    extracted_count = 0
    
    try:
        # Positionner à la frame de début
        cap.set(cv2.CAP_PROP_POS_FRAMES, start_frame)
        
        # ✅ CORRECTION: Utiliser le mapping (frame_originale, index_séquentiel)
        for frame_idx in range(start_frame, end_frame + 1):
            ret, frame = cap.read()
            if not ret:
                break
            
            # Extraire selon l'intervalle
            if (frame_idx - start_frame) % frame_interval == 0:
                # Trouver l'index séquentiel pour cette frame
                sequential_idx = (frame_idx - start_frame) // frame_interval
                filename = frames_dir / f"{sequential_idx:05d}.jpg"  # ← Nommage séquentiel
                
                cv2.imwrite(str(filename), frame, [cv2.IMWRITE_JPEG_QUALITY, 95])
                extracted_count += 1
                
                if extracted_count % 10 == 0:
                    progress = ((frame_idx - start_frame) / (end_frame - start_frame)) * 100
                    print(f"📊 Progrès: {extracted_count} frames extraites ({progress:.1f}%)")
    
    finally:
        cap.release()
    
    print(f"✅ {extracted_count} frames du segment extraites")
    print(f"📋 Nommage: 00000.jpg à {extracted_count-1:05d}.jpg")
    return extracted_count

print("✅ Fonctions de segmentation vidéo chargées!")

✅ Fonctions de segmentation vidéo chargées!


In [6]:
# =============================================================================
# 🎬 EXTRACTION DES FRAMES
# =============================================================================

def extract_frames(video_path: Path, frames_dir: Path, frame_interval: int = 1, force_extraction: bool = False) -> int:
    """Extrait les frames de la vidéo selon l'intervalle spécifié."""

    print(f"🎬 Extraction des frames...")
    print(f"   📹 Source: {video_path}")
    print(f"   📁 Destination: {frames_dir}")
    print(f"   ⏯️  Intervalle: {frame_interval}")
    print(f"   🔄 Force extraction: {'✅ Oui' if force_extraction else '❌ Non'}")

    # Vérification existence du fichier vidéo
    if not video_path.exists():
        raise FileNotFoundError(f"❌ Vidéo non trouvée: {video_path}")

    # Vérification si extraction déjà faite
    existing_frames = list(frames_dir.glob("*.jpg"))
    if existing_frames and not force_extraction:
        print(f"📂 {len(existing_frames)} frames déjà extraites - SKIP")
        return len(existing_frames)
    elif existing_frames and force_extraction:
        print(f"🔄 {len(existing_frames)} frames existantes - SUPPRESSION et ré-extraction...")
        # Supprimer les frames existantes
        for frame_file in existing_frames:
            frame_file.unlink()
        print(f"🗑️  Frames existantes supprimées")

    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise ValueError(f"❌ Impossible d'ouvrir la vidéo: {video_path}")

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    print(f"📊 Vidéo: {total_frames} frames, {fps:.1f} FPS")
    print(f"📊 Frames à extraire: ~{total_frames // frame_interval}")

    extracted_count = 0
    frame_idx = 0

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Extraire seulement selon l'intervalle
            if frame_idx % frame_interval == 0:
                output_idx = frame_idx // frame_interval
                filename = frames_dir / f"{output_idx:05d}.jpg"
                cv2.imwrite(str(filename), frame, [cv2.IMWRITE_JPEG_QUALITY, 95])
                extracted_count += 1

                if extracted_count % 50 == 0:
                    progress = (frame_idx / total_frames) * 100
                    print(f"📊 Progrès: {extracted_count} frames extraites ({progress:.1f}%)")

            frame_idx += 1

    finally:
        cap.release()

    print(f"✅ {extracted_count} frames extraites")
    return extracted_count

def count_existing_frames(frames_dir: Path) -> int:
    """Compte les frames existantes dans le dossier"""
    existing_frames = list(frames_dir.glob("*.jpg"))
    return len(existing_frames)

def extract_frames_with_config(cfg) -> int:
    """Extrait les frames en utilisant la configuration centralisée avec support segmentation"""
    
    # Vérification des fichiers (gestion Colab/Local)
    if cfg.USING_COLAB and not cfg.video_path.exists():
        print("🔬 Mode Colab: Vidéo pas encore uploadée")
        print(f"   💡 Uploadez {cfg.video_path.name} dans {cfg.videos_dir}")
        print(f"   💡 Puis relancez cette cellule ou utilisez cfg.wait_for_files()")
        return 0
    
    # Vérification si extraction activée
    if not cfg.EXTRACT_FRAMES:
        existing_count = count_existing_frames(cfg.frames_dir)
        print(f"⏭️  Extraction désactivée (cfg.EXTRACT_FRAMES = False)")
        if existing_count > 0:
            print(f"📂 Utilisation de {existing_count} frames existantes")
        else:
            print(f"⚠️  Aucune frame trouvée dans {cfg.frames_dir}")
            print(f"💡 Conseil: Activez cfg.EXTRACT_FRAMES = True pour extraire")
        return existing_count
    
    # Décider du mode d'extraction : segmentation ou complet
    if hasattr(cfg, 'SEGMENT_MODE') and cfg.SEGMENT_MODE:
        print("🎯 MODE SEGMENTATION ACTIVÉ")
        
        # Vérifier si project_config est disponible
        if 'project_config' not in globals() or project_config is None:
            print("❌ project_config non disponible pour la segmentation")
            print("💡 Chargez d'abord la configuration projet ou utilisez le mode complet")
            print("🔄 Utilisation du mode complet par défaut...")
            return extract_frames(
                video_path=cfg.video_path,
                frames_dir=cfg.frames_dir,
                frame_interval=cfg.FRAME_INTERVAL,
                force_extraction=cfg.FORCE_EXTRACTION
            )
        
        # Récupérer la frame de référence
        reference_frame = project_config['initial_annotations'][0].get('frame', 0)
        
        # Obtenir les informations vidéo
        cap = cv2.VideoCapture(str(cfg.video_path))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        cap.release()
        
        # ✅ AMÉLIORATION: Utiliser les offsets calculés depuis les secondes
        offset_before_frames, offset_after_frames = cfg.get_segment_offsets_frames()
        
        # Calculer les bornes du segment
        start_frame, end_frame, processed_start_idx, processed_end_idx = calculate_segment_bounds(
            reference_frame=reference_frame,
            offset_before=offset_before_frames,
            offset_after=offset_after_frames,
            total_frames=total_frames,
            frame_interval=cfg.FRAME_INTERVAL
        )
        
        # Extraire les frames du segment
        extracted_count = extract_segment_frames(
            video_path=cfg.video_path,
            frames_dir=cfg.frames_dir,
            start_frame=start_frame,
            end_frame=end_frame,
            frame_interval=cfg.FRAME_INTERVAL,
            force_extraction=cfg.FORCE_EXTRACTION
        )
        
        print(f"\n🎯 SEGMENTATION TERMINÉE:")
        print(f"   📊 Frames du segment: {end_frame - start_frame + 1}")
        print(f"   🎬 Plage: {start_frame} à {end_frame}")
        print(f"   📍 Frame de référence: {reference_frame}")
        print(f"   📊 Frames extraites: {extracted_count}")
        
        return extracted_count
        
    else:
        print("🎬 MODE COMPLET ACTIVÉ (toute la vidéo)")
        return extract_frames(
            video_path=cfg.video_path,
            frames_dir=cfg.frames_dir,
            frame_interval=cfg.FRAME_INTERVAL,
            force_extraction=cfg.FORCE_EXTRACTION
        )

# =============================================================================
# 🚀 EXTRACTION AVEC CONFIGURATION CENTRALISÉE
# =============================================================================

# Extraction intelligente selon l'environnement
print("🎬 Démarrage de l'extraction des frames...")

# Vérification prérequis selon l'environnement
if cfg.USING_COLAB:
    print("🔬 Mode Colab: Vérification des fichiers uploadés...")
    if not cfg.video_path.exists():
        print(f"⚠️  Vidéo pas encore uploadée: {cfg.video_path.name}")
        print(f"   📤 Uploadez dans: {cfg.videos_dir}")
        print(f"   🔄 Puis relancez cette cellule")
        extracted_frames_count = 0
    else:
        extracted_frames_count = extract_frames_with_config(cfg)
else:
    print("🖥️ Mode Local: Extraction directe")
    extracted_frames_count = extract_frames_with_config(cfg)

# Ajout du résultat à la configuration pour usage ultérieur
cfg.extracted_frames_count = extracted_frames_count

print(f"\n📊 RÉSUMÉ EXTRACTION:")
print(f"   🎬 Frames extraites/comptées: {extracted_frames_count}")
print(f"   📁 Dossier: {cfg.frames_dir}")
print(f"   ✅ cfg.extracted_frames_count = {extracted_frames_count}")

🎬 Démarrage de l'extraction des frames...
🖥️ Mode Local: Extraction directe
🎯 MODE SEGMENTATION ACTIVÉ
🎯 CALCUL DES BORNES DE SEGMENTATION:
   📍 Frame de référence: 280
   📉 Offset avant: 50 frames
   📈 Offset après: 50 frames
   🎬 Segment original: frames 230 à 330
   🎬 Segment traité: indices 76 à 110
   📊 Nombre de frames à traiter: 35
🎬 EXTRACTION DU SEGMENT:
   📹 Vidéo: ..\data\videos\SD_13_06_2025_cam1_PdB_S1_T959s_1.mp4
   📁 Destination: ..\data\videos\outputs\SD_13_06_2025_cam1_PdB_S1_T959s_1\frames
   🎯 Segment: frames 230 à 330
   ⏯️  Intervalle: 3
🔄 34 frames existantes - SUPPRESSION et ré-extraction...
🗑️  Frames existantes supprimées
📊 Vidéo: 624 frames total, 25.0 FPS
📊 Nommage: séquentiel de 00000.jpg à 00033.jpg
📊 Progrès: 10 frames extraites (27.0%)
📊 Progrès: 20 frames extraites (57.0%)
📊 Progrès: 30 frames extraites (87.0%)
✅ 34 frames du segment extraites
📋 Nommage: 00000.jpg à 00033.jpg

🎯 SEGMENTATION TERMINÉE:
   📊 Frames du segment: 101
   🎬 Plage: 230 à 330
   

In [7]:
# =============================================================================
# 🤖 INITIALISATION SAM2
# =============================================================================

def initialize_sam2_predictor(cfg, verbose=True):
    """Initialise le predictor SAM2 avec la configuration centralisée"""
    
    # Import SAM2
    from sam2.build_sam import build_sam2_video_predictor
    
    if verbose:
        print(f"🤖 Initialisation SAM2...")
        print(f"   🧠 Modèle: {cfg.model_config_path}")
        print(f"   💾 Checkpoint: {cfg.checkpoint_path}")
        print(f"   🖥️  Device: {cfg.device}")
    
    # Vérification du checkpoint
    if not cfg.checkpoint_path.exists():
        raise FileNotFoundError(f"❌ Checkpoint SAM2 non trouvé: {cfg.checkpoint_path}")
    
    # Construction du predictor
    predictor = build_sam2_video_predictor(
        config_file=cfg.model_config_path,
        ckpt_path=str(cfg.checkpoint_path),
        device=cfg.device
    )
    
    return predictor

def initialize_inference_state(predictor, cfg, verbose=True):
    """Initialise l'état d'inférence avec vérifications"""
    
    if verbose:
        print(f"\n🎬 Initialisation état d'inférence...")
        print(f"   📁 Frames: {cfg.frames_dir}")
    
    # Vérification des frames
    if cfg.extracted_frames_count == 0:
        raise ValueError(f"❌ Aucune frame extraite. Extrayez d'abord les frames.")
    
    # Initialisation de l'état d'inférence
    inference_state = predictor.init_state(
        video_path=str(cfg.frames_dir),
        offload_video_to_cpu=True,    # Économise la mémoire GPU
        offload_state_to_cpu=False    # Garde l'état en GPU
    )
    
    # Reset de l'état
    predictor.reset_state(inference_state)
    
    # Vérification
    loaded_frames = inference_state["num_frames"]
    
    if verbose:
        print(f"\n✅ SAM2 initialisé:")
        print(f"   🖼️  Frames extraites: {cfg.extracted_frames_count}")
        print(f"   🎬 Frames chargées: {loaded_frames}")
        print(f"   ✅ Correspondance: {'OK' if cfg.extracted_frames_count == loaded_frames else 'ERREUR'}")
        
        if cfg.device.type == "cuda":
            allocated = torch.cuda.memory_allocated() / 1024**3
            print(f"   💾 GPU Memory: {allocated:.2f}GB")
    
    if cfg.extracted_frames_count != loaded_frames:
        print(f"⚠️ Incohérence frames : {cfg.extracted_frames_count} extraites vs {loaded_frames} chargées")
    
    return inference_state

# =============================================================================
# 🚀 INITIALISATION AVEC CONFIGURATION CENTRALISÉE
# =============================================================================

# Vérification prérequis
if not hasattr(cfg, 'extracted_frames_count') or cfg.extracted_frames_count == 0:
    print("❌ Frames non extraites. Exécutez d'abord la cellule d'extraction des frames.")
    print("💡 Ou définissez cfg.extracted_frames_count manuellement si frames déjà présentes")
else:
    # Initialisation SAM2
    predictor = initialize_sam2_predictor(cfg)
    inference_state = initialize_inference_state(predictor, cfg)
    
    # Ajout à la configuration pour usage ultérieur
    cfg.predictor = predictor
    cfg.inference_state = inference_state
    
    print("\n✅ SAM2 prêt pour l'ajout d'annotations!")

🤖 Initialisation SAM2...
   🧠 Modèle: configs/sam2.1/sam2.1_hiera_l.yaml
   💾 Checkpoint: ..\checkpoints\sam2.1_hiera_large.pt
   🖥️  Device: cuda

🎬 Initialisation état d'inférence...
   📁 Frames: ..\data\videos\outputs\SD_13_06_2025_cam1_PdB_S1_T959s_1\frames


frame loading (JPEG): 100%|██████████| 34/34 [00:02<00:00, 16.18it/s]



✅ SAM2 initialisé:
   🖼️  Frames extraites: 34
   🎬 Frames chargées: 34
   ✅ Correspondance: OK
   💾 GPU Memory: 1.45GB

✅ SAM2 prêt pour l'ajout d'annotations!


In [8]:
# =============================================================================
# 🎯 AJOUT DES ANNOTATIONS INITIALES
# =============================================================================
def add_initial_annotations(predictor, inference_state, project_config: Dict[str, Any], frame_interval: int = 1, segment_info: Dict = None):
    """Ajoute les annotations initiales depuis la configuration projet avec conversion des frames et support segmentation."""

    print(f"🎯 Ajout des annotations initiales...")

    # Création du mapping obj_id -> obj_type
    obj_types = {}
    for obj in project_config['objects']:
        obj_types[obj['obj_id']] = obj['obj_type']

    # Extraction automatique des annotations et frames depuis le JSON
    all_annotations = []
    annotation_frames = []

    for frame_data in project_config['initial_annotations']:
        frame_idx_original = frame_data['frame']  # ← Frame originale du JSON
        
        # ✅ CORRECTION: Conversion avec prise en compte de la segmentation
        frame_idx_processed = frame_idx_original // frame_interval
        
        # 🎯 NOUVEAU: Ajustement pour le mode segmentation
        if segment_info:
            # Calculer l'offset par rapport au début du segment
            segment_start_processed = segment_info['start_frame'] // frame_interval
            frame_idx_segment = frame_idx_processed - segment_start_processed
            
            print(f"   🎯 MODE SEGMENTATION:")
            print(f"      📍 Frame originale: {frame_idx_original}")
            print(f"      📍 Frame traitée: {frame_idx_processed}")
            print(f"      📍 Segment commence à: {segment_start_processed} (traité)")
            print(f"      📍 Index dans le segment: {frame_idx_segment}")
            
            # Vérifier que la frame est dans le segment
            segment_end_processed = segment_info['end_frame'] // frame_interval
            if frame_idx_processed < segment_start_processed or frame_idx_processed > segment_end_processed:
                raise ValueError(f"❌ Frame d'annotation {frame_idx_processed} en dehors du segment [{segment_start_processed}, {segment_end_processed}]")
            
            # Utiliser l'index dans le segment
            frame_idx_for_sam = frame_idx_segment
        else:
            frame_idx_for_sam = frame_idx_processed
        
        annotations = frame_data['annotations']
        annotation_frames.append(frame_idx_for_sam)

        print(f"   📍 Frame {frame_idx_original} (original) → {frame_idx_for_sam} (pour SAM2): {len(annotations)} annotations")

        for annotation in annotations:
            all_annotations.append({
                'frame_original': frame_idx_original,
                'frame_processed': frame_idx_processed,
                'frame_for_sam': frame_idx_for_sam,  # ← Nouvel index pour SAM2
                'obj_id': annotation['obj_id'],
                'points': annotation['points'],
                'obj_type': obj_types.get(annotation['obj_id'], f'unknown_{annotation["obj_id"]}')
            })

    if not all_annotations:
        raise ValueError(f"❌ Aucune annotation trouvée dans le fichier config")

    print(f"   📊 Total: {len(all_annotations)} annotations sur {len(set(annotation_frames))} frames")

    # Ajout des annotations à SAM2 avec les frames converties
    added_objects = []

    for annotation_data in all_annotations:
        frame_idx_for_sam = annotation_data['frame_for_sam']  # ← Utiliser l'index ajusté
        frame_idx_original = annotation_data['frame_original']
        obj_id = annotation_data['obj_id']
        obj_type = annotation_data['obj_type']
        points_data = annotation_data['points']

        # Extraction des coordonnées et labels
        points = np.array([[p['x'], p['y']] for p in points_data], dtype=np.float32)
        labels = np.array([p['label'] for p in points_data], dtype=np.int32)

        print(f"   🎯 Frame {frame_idx_original}→{frame_idx_for_sam} - Objet {obj_id} ({obj_type}): {len(points)} points à ({points[0][0]:.0f}, {points[0][1]:.0f})")

        # ✅ CORRECTION: Utiliser frame_idx_for_sam (ajusté pour le segment)
        _, out_obj_ids, out_mask_logits = predictor.add_new_points_or_box(
            inference_state,
            frame_idx_for_sam,  # ← Frame ajustée pour SAM2
            obj_id,
            points,
            labels
        )

        # Éviter les doublons dans added_objects
        if not any(obj['obj_id'] == obj_id for obj in added_objects):
            added_objects.append({
                'obj_id': obj_id,
                'obj_type': obj_type,
                'points_count': len(points)
            })

    # Vérification
    sam_obj_ids = inference_state["obj_ids"]

    print(f"\n📊 RÉSUMÉ ANNOTATIONS:")
    print(f"   🎯 Annotations configurées: {len(all_annotations)}")
    print(f"   🎯 Objets uniques: {len(added_objects)}")
    print(f"   ✅ Objets ajoutés à SAM2: {len(sam_obj_ids)}")
    print(f"   🆔 IDs: {sorted(sam_obj_ids)}")
    
    if segment_info:
        print(f"   🎯 Mode segmentation: Frame d'annotation ajustée à l'index {annotation_data['frame_for_sam']} dans le segment")
    else:
        print(f"   🎬 Mode complet: Frames utilisées (original→traitée): {[(a['frame_original'], a['frame_processed']) for a in all_annotations[:3]]}...")

    # Résumé par type
    type_counts = {}
    for obj in added_objects:
        obj_type = obj['obj_type']
        type_counts[obj_type] = type_counts.get(obj_type, 0) + 1
    print(f"   🏷️  Types: {dict(type_counts)}")

    return added_objects, all_annotations

# =============================================================================
# 🚀 AJOUT AVEC CONFIGURATION CENTRALISÉE CORRIGÉE
# =============================================================================

# Vérifications prérequis
if not hasattr(cfg, 'predictor') or not hasattr(cfg, 'inference_state'):
    print("❌ SAM2 non initialisé. Exécutez d'abord la cellule d'initialisation SAM2.")
elif project_config is None:
    print("❌ Configuration projet non chargée.")
    if cfg.USING_COLAB:
        print("💡 Sur Colab, utilisez: project_config = wait_and_load_config(cfg)")
    else:
        print("💡 La configuration devrait être chargée automatiquement")
else:
    # 🎯 NOUVEAU: Préparer les informations de segmentation si applicable
    segment_info = None
    if hasattr(cfg, 'SEGMENT_MODE') and cfg.SEGMENT_MODE:
        # Récupérer les informations du segment depuis la configuration
        reference_frame = project_config['initial_annotations'][0].get('frame', 0)
        
        # Obtenir les informations vidéo
        cap = cv2.VideoCapture(str(cfg.video_path))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        cap.release()
        
        # ✅ CORRECTION: Utiliser les offsets convertis en frames entières
        offset_before_frames, offset_after_frames = cfg.get_segment_offsets_frames()
        
        # Calculer les bornes du segment
        start_frame, end_frame, processed_start_idx, processed_end_idx = calculate_segment_bounds(
            reference_frame=reference_frame,
            offset_before=offset_before_frames,
            offset_after=offset_after_frames,
            total_frames=total_frames,
            frame_interval=cfg.FRAME_INTERVAL
        )
        
        segment_info = {
            'start_frame': start_frame,
            'end_frame': end_frame,
            'processed_start_idx': processed_start_idx,
            'processed_end_idx': processed_end_idx
        }
        
        print("🎯 Informations segmentation préparées pour l'ajout d'annotations")
    
    # ✅ CORRECTION: Passer les informations de segmentation
    added_objects, initial_annotations_data = add_initial_annotations(
        cfg.predictor, 
        cfg.inference_state, 
        project_config,
        cfg.FRAME_INTERVAL,
        segment_info  # ← Nouvelles informations de segmentation
    )
    
    # Ajout à la configuration pour usage ultérieur
    cfg.added_objects = added_objects
    cfg.initial_annotations_data = initial_annotations_data
    
    print("\n✅ Annotations initiales ajoutées avec succès!")

🎯 CALCUL DES BORNES DE SEGMENTATION:
   📍 Frame de référence: 280
   📉 Offset avant: 50 frames
   📈 Offset après: 50 frames
   🎬 Segment original: frames 230 à 330
   🎬 Segment traité: indices 76 à 110
   📊 Nombre de frames à traiter: 35
🎯 Informations segmentation préparées pour l'ajout d'annotations
🎯 Ajout des annotations initiales...
   🎯 MODE SEGMENTATION:
      📍 Frame originale: 280
      📍 Frame traitée: 93
      📍 Segment commence à: 76 (traité)
      📍 Index dans le segment: 17
   📍 Frame 280 (original) → 17 (pour SAM2): 13 annotations
   📊 Total: 13 annotations sur 1 frames
   🎯 Frame 280→17 - Objet 1 (player): 1 points à (572, 444)
   🎯 Frame 280→17 - Objet 2 (player): 1 points à (604, 321)
   🎯 Frame 280→17 - Objet 3 (player): 1 points à (861, 325)
   🎯 Frame 280→17 - Objet 4 (player): 1 points à (1324, 328)
   🎯 Frame 280→17 - Objet 5 (player): 1 points à (1022, 238)
   🎯 Frame 280→17 - Objet 6 (player): 1 points à (778, 289)
   🎯 Frame 280→17 - Objet 7 (player): 1 points

c:\Users\antoi\Documents\Work_Learn\Stage-Rennes\EVA2SPORT\.venv\lib\site-packages\sam2\sam2_video_predictor.py:786: UserWarning: cannot import name '_C' from 'sam2' (c:\Users\antoi\Documents\Work_Learn\Stage-Rennes\EVA2SPORT\.venv\lib\site-packages\sam2\__init__.py)

Skipping the post-processing step due to the error above. You can still use SAM 2 and it's OK to ignore the error above, although some post-processing functionality may be limited (which doesn't affect the results in most cases; see https://github.com/facebookresearch/sam2/blob/main/INSTALL.md).
  pred_masks_gpu = fill_holes_in_mask_scores(


In [9]:
# =============================================================================
# 🔧 FONCTIONS UTILITAIRES POUR ANNOTATIONS COMPLÈTES
# =============================================================================

def get_object_scores(predictor, inference_state, frame_idx, obj_id):
    """Récupère les scores d'objet de manière propre et sûre"""
    try:
        obj_idx = predictor._obj_id_to_idx(inference_state, obj_id)
        obj_output_dict = inference_state["output_dict_per_obj"][obj_idx]
        temp_output_dict = inference_state["temp_output_dict_per_obj"][obj_idx]

        # Chercher dans les outputs
        frame_output = None
        if frame_idx in temp_output_dict["cond_frame_outputs"]:
            frame_output = temp_output_dict["cond_frame_outputs"][frame_idx]
        elif frame_idx in temp_output_dict["non_cond_frame_outputs"]:
            frame_output = temp_output_dict["non_cond_frame_outputs"][frame_idx]
        elif frame_idx in obj_output_dict["cond_frame_outputs"]:
            frame_output = obj_output_dict["cond_frame_outputs"][frame_idx]
        elif frame_idx in obj_output_dict["non_cond_frame_outputs"]:
            frame_output = obj_output_dict["non_cond_frame_outputs"][frame_idx]

        if frame_output and "object_score_logits" in frame_output:
            object_score_logits = frame_output["object_score_logits"]
            return torch.sigmoid(object_score_logits).item()
        return None

    except Exception as e:
        return None

def calculate_bbox_from_rle(rle_data: Dict[str, Any]) -> Optional[Dict[str, int]]:
    """Calcule la bounding box depuis un RLE base64."""
    from pycocotools.mask import toBbox

    try:
        rle = {
            "size": rle_data["size"],
            "counts": base64.b64decode(rle_data["counts"])
        }

        bbox = toBbox(rle)

        result = {
            "x": int(bbox[0]),
            "y": int(bbox[1]),
            "width": int(bbox[2]),
            "height": int(bbox[3])
        }
        return result

    except Exception as e:
        print(f"⚠️ Erreur calcul bbox: {e}")
        return None

def image_to_world(point_2d, cam_params):
    """
    Projette un point 2D de l'image vers le plan du terrain (Z=0).
    """
    # Create projection matrix P
    K = np.array([
        [cam_params["cam_params"]["x_focal_length"], 0, cam_params["cam_params"]["principal_point"][0]],
        [0, cam_params["cam_params"]["y_focal_length"], cam_params["cam_params"]["principal_point"][1]],
        [0, 0, 1]
    ])
    R = np.array(cam_params["cam_params"]["rotation_matrix"])
    t = -R @ np.array(cam_params["cam_params"]["position_meters"])
    P = K @ np.hstack((R, t.reshape(-1,1)))

    # Create point on image plane in homogeneous coordinates
    point_2d_h = np.array([point_2d[0], point_2d[1], 1])

    # Back-project ray from camera
    ray = np.linalg.inv(K) @ point_2d_h
    ray = R.T @ ray

    # Find intersection with Z=0 plane
    camera_pos = np.array(cam_params["cam_params"]["position_meters"])
    t = -camera_pos[2] / ray[2]
    world_point = camera_pos + t * ray

    return world_point[:2]  # Return only X,Y coordinates since Z=0

def calculate_points_output(bbox_output: dict, cam_params: dict = None) -> dict:
    """
    Calcule les points de sortie à partir de la bbox output.

    Args:
        bbox_output: Dict avec 'x', 'y', 'width', 'height'
        cam_params: Paramètres de calibration caméra pour projection terrain

    Returns:
        Dict avec les points calculés séparés par plan (image vs field)
    """
    if not bbox_output:
        return None

    # Calculer le point CENTER_BOTTOM dans le plan image
    center_bottom_x = bbox_output['x'] + bbox_output['width'] / 2
    center_bottom_y = bbox_output['y'] + bbox_output['height']  # Bas de la bbox

    # Structure avec séparation image/field
    points_output = {
        "image": {
            "CENTER_BOTTOM": {
                "x": float(center_bottom_x),
                "y": float(center_bottom_y)
            }
        },
        "field": {
            "CENTER_BOTTOM": None
        }
    }

    # Projection vers le terrain si les paramètres caméra sont fournis
    if cam_params:
        try:
            # Projeter le point CENTER_BOTTOM vers le terrain
            image_point = [center_bottom_x, center_bottom_y]
            field_point = image_to_world(image_point, cam_params)

            points_output["field"]["CENTER_BOTTOM"] = {
                "x": float(field_point[0]),
                "y": float(field_point[1])
            }

        except Exception as e:
            print(f"⚠️ Erreur projection terrain: {e}")
            points_output["field"]["CENTER_BOTTOM"] = None

    return points_output

def create_mask_annotation(obj_id: int, mask_logits, predictor=None, inference_state=None,
                         frame_idx=None, cam_params: Dict = None) -> Dict:
    """
    Crée une annotation de masque complète avec points input/output, bbox et scores.
    """
    # Conversion en masque binaire
    mask = (mask_logits > 0.0).cpu().numpy()
    if mask.ndim == 3 and mask.shape[0] == 1:
        mask = np.squeeze(mask, axis=0)

    # Encodage RLE
    if not mask.flags['F_CONTIGUOUS']:
        mask = np.asfortranarray(mask)

    rle = encode_rle(mask.astype(np.uint8))
    base64_counts = base64.b64encode(rle["counts"]).decode('ascii')

    # Calcul bbox et points output si masque non vide
    bbox_output = None
    points_output = None

    if mask.sum() > 0:
        from pycocotools.mask import toBbox
        bbox = toBbox(rle)
        bbox_output = {
            "x": int(bbox[0]),
            "y": int(bbox[1]),
            "width": int(bbox[2]),
            "height": int(bbox[3])
        }
        # Calcul des points output depuis la bbox
        points_output = calculate_points_output(bbox_output, cam_params)

    # Récupération du score du masque
    mask_score = None

    if predictor and inference_state and frame_idx is not None:
        # Score du masque
        mask_score = get_object_scores(predictor, inference_state, frame_idx, obj_id)

    # Structure d'annotation complète
    return {
        "id": str(uuid.uuid4()),
        "objectId": str(obj_id),
        "type": "mask",
        "mask": {
            "format": "rle_coco_base64",
            "size": [int(rle["size"][0]), int(rle["size"][1])],
            "counts": base64_counts
        },
        "bbox": {
            "output": bbox_output
        },
        "points": {
            "output": points_output
        },
        "maskScore": mask_score,
        "pose": None,
        "warning": False
    }

print("✅ Fonctions utilitaires pour annotations chargées!")

✅ Fonctions utilitaires pour annotations chargées!


In [10]:
# =============================================================================
# 🔄 PROPAGATION ET GÉNÉRATION DES ANNOTATIONS
# =============================================================================
def generate_frame_mapping_with_anchor(total_frames: int, frame_interval: int, anchor_frame: int) -> List[Optional[int]]:
    """
    Génère le mapping en s'assurant que anchor_frame est incluse.
    
    Args:
        total_frames: Nombre total de frames dans la vidéo originale
        frame_interval: Intervalle entre frames (ex: 10 = 1 frame sur 10)
        anchor_frame: Frame qui DOIT être incluse (frame d'annotation initiale)
    
    Returns:
        Liste où l'index = frame originale, valeur = frame traitée (ou None si pas traitée)
    """
    frame_mapping = [None] * total_frames
    processed_frames = set()
    processed_idx = 0
    
    # 1. S'assurer que anchor_frame est incluse
    if 0 <= anchor_frame < total_frames:
        frame_mapping[anchor_frame] = processed_idx
        processed_frames.add(anchor_frame)
        processed_idx += 1
    
    # 2. Ajouter les frames selon l'intervalle, en évitant les doublons
    for original_idx in range(0, total_frames, frame_interval):
        if original_idx not in processed_frames:
            frame_mapping[original_idx] = processed_idx
            processed_frames.add(original_idx)
            processed_idx += 1
    
    # 3. Réorganiser les indices pour que anchor_frame ait l'index correspondant à sa position chronologique
    # Tri des frames traitées par ordre chronologique
    sorted_frames = sorted(processed_frames)
    final_mapping = [None] * total_frames
    
    for new_idx, original_frame in enumerate(sorted_frames):
        final_mapping[original_frame] = new_idx
    
    return final_mapping, sorted_frames

def get_initial_annotation_frame(project_config: Dict[str, Any], frame_interval: int = 1) -> int:
    """Récupère la frame d'annotation initiale depuis la config et la convertit selon l'intervalle"""
    if not project_config.get('initial_annotations'):
        return 0
    
    # Prendre la première frame d'annotation comme anchor (frame originale)
    first_annotation = project_config['initial_annotations'][0]
    original_frame = first_annotation.get('frame', 0)
    
    # ✅ CONVERSION: Frame originale → Frame traitée selon l'intervalle
    processed_frame = original_frame // frame_interval
    
    print(f"   🔄 Conversion anchor frame: {original_frame} (original) → {processed_frame} (traitée avec intervalle {frame_interval})")
    
    return processed_frame

def get_video_info(video_path: Path) -> Dict[str, Any]:
    """Récupère les informations de la vidéo"""
    cap = cv2.VideoCapture(str(video_path))
    if not cap.isOpened():
        raise ValueError(f"❌ Impossible d'ouvrir la vidéo: {video_path}")
    
    video_info = {
        'total_frames': int(cap.get(cv2.CAP_PROP_FRAME_COUNT)),
        'fps': cap.get(cv2.CAP_PROP_FPS),
        'width': int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),
        'height': int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    }
    cap.release()
    return video_info
def create_project_structure_with_config(cfg, project_config: Dict[str, Any], added_objects: List[Dict]) -> Dict[str, Any]:
    """Crée la structure JSON du projet avec la configuration centralisée."""
    
    # Informations vidéo
    video_info = get_video_info(cfg.video_path)
    
    # ✅ CORRECTION: Passer l'intervalle pour la conversion
    anchor_frame = get_initial_annotation_frame(project_config, cfg.FRAME_INTERVAL)
    
    # Calcul du mapping avec la frame déjà convertie
    frame_mapping, processed_frames = generate_frame_mapping_with_anchor(
        video_info['total_frames'], 
        cfg.FRAME_INTERVAL,
        anchor_frame * cfg.FRAME_INTERVAL  # ← Reconvertir pour le mapping original
    )
    processed_frame_count = len(processed_frames)

    # Structure objects avec couleurs
    import random
    import colorsys

    config_objects_mapping = {}
    for obj in project_config['objects']:
        config_objects_mapping[obj['obj_id']] = obj

    objects = {}
    for obj_data in added_objects:
        obj_id = str(obj_data['obj_id'])
        obj_type = obj_data['obj_type']

        # Récupérer les informations complètes depuis le config
        config_obj = config_objects_mapping.get(int(obj_id), {})

        # Couleur aléatoire reproductible
        random.seed(int(obj_id) * 12345)
        hue = random.random()
        rgb = colorsys.hsv_to_rgb(hue, 0.8, 0.9)
        hex_color = "#{:02x}{:02x}{:02x}".format(
            int(rgb[0] * 255), int(rgb[1] * 255), int(rgb[2] * 255)
        )

        objects[obj_id] = {
            "id": obj_id,
            "type": obj_type,
            "team": config_obj.get('team', None),
            "jersey_number": config_obj.get('jersey_number', None),
            "jersey_color": config_obj.get('jersey_color', None),
            "role": config_obj.get('role', None),
            "display_color": hex_color
        }

    return {
        "format_version": "1.0",
        "video": f"{cfg.VIDEO_NAME}.mp4",
        "metadata": {
            "project_id": str(uuid.uuid4()),
            "created_at": datetime.now().isoformat() + "Z",
            "fps": video_info['fps'],
            "resolution": {
                "width": video_info['width'],
                "height": video_info['height'],
                "aspect_ratio": round(video_info['width'] / video_info['height'], 2)
            },
            "frame_interval": cfg.FRAME_INTERVAL,
            "frame_count_original": video_info['total_frames'],
            "frame_count_processed": processed_frame_count,
            "frame_mapping": frame_mapping,
            "anchor_frame": anchor_frame,  # ✅ AJOUT: Stockage de la frame d'ancrage
            "static_video": False
        },
        "calibration": project_config['calibration'],
        "objects": objects,
        "initial_annotations": project_config['initial_annotations'],
        "annotations": {}
    }

def run_sam2_propagation(cfg, project_config):
    """Exécute la propagation SAM2 et génère toutes les annotations"""
    
    print(f"🔄 Démarrage de la propagation...")
    print(f"   🎬 {cfg.extracted_frames_count} frames à traiter")
    print(f"   🎯 {len(cfg.added_objects)} objets à suivre")

    # Création de la structure du projet
    project = create_project_structure_with_config(cfg, project_config, cfg.added_objects)

    # Propagation et annotation
    frame_count = 0
    for out_frame_idx, out_obj_ids, out_mask_logits in cfg.predictor.propagate_in_video(cfg.inference_state):

        if str(out_frame_idx) not in project['annotations']:
            project['annotations'][str(out_frame_idx)] = []

        for i, out_obj_id in enumerate(out_obj_ids):
            annotation = create_mask_annotation(
                obj_id=out_obj_id,
                mask_logits=out_mask_logits[i],
                predictor=cfg.predictor,
                inference_state=cfg.inference_state,
                frame_idx=out_frame_idx,
                cam_params=project_config['calibration']['camera_parameters']
            )
            project['annotations'][str(out_frame_idx)].append(annotation)
        
        frame_count += 1
        if frame_count % 50 == 0:
            progress = (frame_count / cfg.extracted_frames_count) * 100
            print(f"   📊 Progrès: {frame_count}/{cfg.extracted_frames_count} frames ({progress:.1f}%)")
    
    print(f"✅ Propagation terminée: {frame_count} frames traitées")
    return project


In [11]:
def run_sam2_bidirectional_propagation(cfg, project_config):
    """Exécute la propagation SAM2 bidirectionnelle depuis la frame d'annotation avec support segmentation"""
    
    print(f"🔄 Démarrage de la propagation bidirectionnelle...")
    
    # 🎯 NOUVEAU: Gestion du mode segmentation
    segment_mode = hasattr(cfg, 'SEGMENT_MODE') and cfg.SEGMENT_MODE
    
    if segment_mode:
        print(f"🎯 Mode segmentation activé")
        
        # En mode segmentation, utiliser les indices ajustés
        reference_frame_original = project_config['initial_annotations'][0].get('frame', 0)
        
        # Recalculer les bornes du segment
        cap = cv2.VideoCapture(str(cfg.video_path))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        cap.release()
        
        # ✅ AMÉLIORATION: Utiliser les offsets calculés depuis les secondes
        offset_before_frames, offset_after_frames = cfg.get_segment_offsets_frames()
        
        start_frame, end_frame, processed_start_idx, processed_end_idx = calculate_segment_bounds(
            reference_frame=reference_frame_original,
            offset_before=offset_before_frames,
            offset_after=offset_after_frames,
            total_frames=total_frames,
            frame_interval=cfg.FRAME_INTERVAL
        )
        
        # Calculer l'index dans le segment
        reference_frame_processed = reference_frame_original // cfg.FRAME_INTERVAL
        segment_start_processed = start_frame // cfg.FRAME_INTERVAL
        anchor_frame_in_segment = reference_frame_processed - segment_start_processed
        
        print(f"   📍 Frame de référence originale: {reference_frame_original}")
        print(f"   📍 Frame de référence traitée: {reference_frame_processed}")
        print(f"   📍 Segment commence à: {segment_start_processed} (traité)")
        print(f"   📍 Index dans le segment: {anchor_frame_in_segment}")
        print(f"   📊 Frames disponibles dans le segment: 0 à {cfg.extracted_frames_count - 1}")
        
        # Vérifier que l'index est valide
        if anchor_frame_in_segment < 0 or anchor_frame_in_segment >= cfg.extracted_frames_count:
            raise ValueError(f"❌ Index anchor {anchor_frame_in_segment} en dehors des frames disponibles [0, {cfg.extracted_frames_count - 1}]")
        
        # En mode segmentation, les frames vont de 0 à extracted_frames_count-1
        anchor_processed_idx = anchor_frame_in_segment
        total_frames_available = cfg.extracted_frames_count
        
        # Mapping simplifié pour la segmentation
        processed_frames = list(range(total_frames_available))
        frame_mapping = [None] * total_frames
        
        # Remplir le mapping pour le segment
        for i, original_frame in enumerate(range(start_frame, end_frame + 1, cfg.FRAME_INTERVAL)):
            if original_frame < total_frames:
                frame_mapping[original_frame] = i
        
    else:
        print(f"🎬 Mode complet activé")
        
        # Mode complet : logique originale
        reference_frame_original = project_config['initial_annotations'][0].get('frame', 0)
        reference_frame_processed = reference_frame_original // cfg.FRAME_INTERVAL
        
        # Informations vidéo et mapping
        video_info = get_video_info(cfg.video_path)
        frame_mapping, processed_frames = generate_frame_mapping_with_anchor(
            video_info['total_frames'], 
            cfg.FRAME_INTERVAL, 
            reference_frame_processed
        )
        
        # Trouver l'index de la frame d'ancrage dans les frames traitées
        anchor_processed_idx = frame_mapping[reference_frame_processed]
        total_frames_available = len(processed_frames)
        
        print(f"   📍 Frame d'ancrage: {reference_frame_original} (original) → {reference_frame_processed} (traitée) → {anchor_processed_idx} (index)")
        print(f"   📊 Frames disponibles: 0 à {total_frames_available - 1}")
    
    print(f"   📍 Anchor frame index: {anchor_processed_idx}")
    print(f"   📊 Total frames disponibles: {total_frames_available}")
    
    # Création de la structure du projet
    project = create_project_structure_with_config(cfg, project_config, cfg.added_objects)
    
    # ✅ CORRECTION: Métadonnées harmonisées pour les deux modes
    project['metadata']['segment_mode'] = segment_mode
    project['metadata']['anchor_processed_idx'] = anchor_processed_idx  # ← Toujours présent
    project['metadata']['frame_mapping'] = frame_mapping
    
    if segment_mode:
        # Métadonnées spécifiques au mode segmentation
        project['metadata']['segment_start_frame'] = start_frame
        project['metadata']['segment_end_frame'] = end_frame
        project['metadata']['segment_reference_frame'] = reference_frame_original
        project['metadata']['anchor_frame_in_segment'] = anchor_frame_in_segment
        project['metadata']['anchor_frame'] = reference_frame_original  # ← Frame originale en mode segmentation
    else:
        # Métadonnées spécifiques au mode complet
        project['metadata']['anchor_frame'] = reference_frame_processed  # ← Frame traitée en mode complet
    
    # === PHASE 1: PROPAGATION INVERSE (anchor → 0) ===
    if anchor_processed_idx > 0:
        print(f"\n🔄 PHASE 1: Propagation inverse (frame {anchor_processed_idx} → 0)")
        
        reverse_frames_count = 0
        for out_frame_idx, out_obj_ids, out_mask_logits in cfg.predictor.propagate_in_video(
            cfg.inference_state,
            start_frame_idx=anchor_processed_idx,
            max_frame_num_to_track=anchor_processed_idx + 1,  # +1 pour inclure l'anchor
            reverse=True
        ):
            # Traitement identique à la propagation normale
            if str(out_frame_idx) not in project['annotations']:
                project['annotations'][str(out_frame_idx)] = []

            for i, out_obj_id in enumerate(out_obj_ids):
                annotation = create_mask_annotation(
                    obj_id=out_obj_id,
                    mask_logits=out_mask_logits[i],
                    predictor=cfg.predictor,
                    inference_state=cfg.inference_state,
                    frame_idx=out_frame_idx,
                    cam_params=project_config['calibration']['camera_parameters']
                )
                project['annotations'][str(out_frame_idx)].append(annotation)
            
            reverse_frames_count += 1
            if reverse_frames_count % 10 == 0:
                print(f"     📊 Frames traitées (reverse): {reverse_frames_count}")
        
        print(f"   ✅ Phase reverse terminée: {reverse_frames_count} frames")
    else:
        print(f"\n⏭️ PHASE 1: Propagation inverse skippée (anchor à la frame 0)")
    
    # === PHASE 2: PROPAGATION AVANT (anchor → fin) ===
    remaining_frames = total_frames_available - anchor_processed_idx
    if remaining_frames > 1:  # > 1 car anchor déjà traitée
        print(f"\n🔄 PHASE 2: Propagation avant (frame {anchor_processed_idx} → {total_frames_available - 1})")
        
        forward_frames_count = 0
        for out_frame_idx, out_obj_ids, out_mask_logits in cfg.predictor.propagate_in_video(
            cfg.inference_state,
            start_frame_idx=anchor_processed_idx,
            max_frame_num_to_track=remaining_frames,
            reverse=False
        ):
            # Éviter de traiter à nouveau l'anchor frame
            if out_frame_idx == anchor_processed_idx and str(out_frame_idx) in project['annotations']:
                continue
                
            if str(out_frame_idx) not in project['annotations']:
                project['annotations'][str(out_frame_idx)] = []

            for i, out_obj_id in enumerate(out_obj_ids):
                annotation = create_mask_annotation(
                    obj_id=out_obj_id,
                    mask_logits=out_mask_logits[i],
                    predictor=cfg.predictor,
                    inference_state=cfg.inference_state,
                    frame_idx=out_frame_idx,
                    cam_params=project_config['calibration']['camera_parameters']
                )
                project['annotations'][str(out_frame_idx)].append(annotation)
            
            forward_frames_count += 1
            if forward_frames_count % 20 == 0:
                progress = (forward_frames_count / remaining_frames) * 100
                print(f"     📊 Frames traitées (forward): {forward_frames_count}/{remaining_frames} ({progress:.1f}%)")
        
        print(f"   ✅ Phase forward terminée: {forward_frames_count} frames")
    else:
        print(f"\n⏭️ PHASE 2: Propagation avant skippée (anchor à la fin)")
    
    total_frames_processed = len(project['annotations'])
    print(f"\n✅ Propagation bidirectionnelle terminée:")
    print(f"   📊 Total frames traitées: {total_frames_processed}")
    if segment_mode:
        print(f"   🎯 Mode segmentation: Frame d'ancrage à l'index {anchor_processed_idx} dans le segment")
        print(f"   📍 Segment original: frames {start_frame} à {end_frame}")
    else:
        print(f"   🎯 Mode complet: Frame d'ancrage {anchor_processed_idx} sur {total_frames_available}")
    
    return project

In [12]:
# ============================================================================
# 🎯 PROPAGATION AVEC SEGMENTATION OPTIONNELLE
# =============================================================================
# Vérifier le mode de segmentation
if hasattr(cfg, 'SEGMENT_MODE') and cfg.SEGMENT_MODE:
    print("🎯 MODE SEGMENTATION ACTIVÉ")
    
    # Récupérer la frame de référence
    reference_frame = project_config['initial_annotations'][0].get('frame', 0)
    
    # Obtenir les informations vidéo
    cap = cv2.VideoCapture(str(cfg.video_path))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    
    # ✅ AMÉLIORATION: Utiliser les offsets calculés depuis les secondes
    offset_before_frames, offset_after_frames = cfg.get_segment_offsets_frames()
    
    # Calculer les bornes du segment
    start_frame, end_frame, processed_start_idx, processed_end_idx = calculate_segment_bounds(
        reference_frame=reference_frame,
        offset_before=offset_before_frames,
        offset_after=offset_after_frames,
        total_frames=total_frames,
        frame_interval=cfg.FRAME_INTERVAL
    )
    
    # Réajuster l'extraction pour le segment
    if cfg.EXTRACT_FRAMES:
        extracted_count = extract_segment_frames(
            video_path=cfg.video_path,
            frames_dir=cfg.frames_dir,
            start_frame=start_frame,
            end_frame=end_frame,
            frame_interval=cfg.FRAME_INTERVAL,
            force_extraction=cfg.FORCE_EXTRACTION
        )
        cfg.extracted_frames_count = extracted_count
        print(f"✅ {extracted_count} frames du segment extraites")
    
    print(f"🎯 SEGMENT CONFIGURÉ:")
    print(f"   📊 Frames du segment: {end_frame - start_frame + 1}")
    print(f"   🎬 Plage: {start_frame} à {end_frame}")
    print(f"   📍 Frame de référence: {reference_frame}")
    
else:
    print("🎬 MODE COMPLET ACTIVÉ (toute la vidéo)")
    
# =============================================================================
# 🔄 PROPAGATION BIDIRECTIONNELLE (REMPLACE L'ANCIENNE PROPAGATION)
# =============================================================================

# Vérifications des prérequis
missing_requirements = []
required_attrs = ['predictor', 'inference_state', 'added_objects', 'extracted_frames_count']

for attr in required_attrs:
    if not hasattr(cfg, attr):
        missing_requirements.append(attr)

if project_config is None:
    missing_requirements.append("project_config")

if missing_requirements:
    print(f"❌ Prérequis manquants: {missing_requirements}")
    print("💡 Exécutez d'abord les cellules précédentes dans l'ordre:")
    print("   1. Configuration centralisée")
    print("   2. Installation SAM2") 
    print("   3. Chargement configuration projet")
    print("   4. Extraction des frames")
    print("   5. Initialisation SAM2")
    print("   6. Ajout des annotations initiales")
else:
    # 🚀 NOUVELLE PROPAGATION BIDIRECTIONNELLE
    print("🎯 Utilisation de la propagation bidirectionnelle améliorée...")
    
    project = run_sam2_bidirectional_propagation(cfg, project_config)
    
    # Ajout à la configuration pour usage ultérieur
    cfg.project = project
    
    # Statistiques finales améliorées
    total_annotations = sum(len(annotations) for annotations in project['annotations'].values())
    unique_frames = len(project['annotations'])
    anchor_frame = project['metadata']['anchor_frame']
    anchor_processed_idx = project['metadata']['anchor_processed_idx']
    
    print(f"\n📊 RÉSULTATS PROPAGATION BIDIRECTIONNELLE:")
    print(f"   🎯 Frame d'ancrage: {anchor_frame} (original) → {anchor_processed_idx} (traitée)")
    print(f"   🎬 Frames originales: {project['metadata']['frame_count_original']:,}")
    print(f"   🎬 Frames traitées: {project['metadata']['frame_count_processed']:,}")
    print(f"   📍 Frames avec annotations: {unique_frames:,}")
    print(f"   📍 Annotations totales: {total_annotations:,}")
    print(f"   🎯 Objets suivis: {len(project['objects'])}")
    print(f"   ⏯️  Intervalle frames: {project['metadata']['frame_interval']}")
    
    # Affichage du mapping frame pour vérification
    frame_mapping = project['metadata']['frame_mapping']
    mapped_frames = [(i, v) for i, v in enumerate(frame_mapping) if v is not None]
    sample_mapping = mapped_frames[:10] if len(mapped_frames) > 10 else mapped_frames
    
    print(f"\n🗂️  MAPPING FRAMES (échantillon):")
    print(f"   📋 Format: (frame_originale → frame_traitée)")
    print(f"   📋 Échantillon: {sample_mapping}")
    if len(mapped_frames) > 10:
        print(f"   📋 ... et {len(mapped_frames) - 10} autres frames")
    
    # Vérification que l'anchor frame est bien dans le mapping
    if anchor_frame < len(frame_mapping) and frame_mapping[anchor_frame] is not None:
        print(f"   ✅ Frame d'ancrage {anchor_frame} correctement mappée → {frame_mapping[anchor_frame]}")
    else:
        print(f"   ⚠️  Attention: Frame d'ancrage {anchor_frame} non trouvée dans le mapping")
    
    # Résumé par type d'objet
    if project['objects']:
        type_counts = {}
        for obj_data in project['objects'].values():
            obj_type = obj_data.get('type', 'unknown')
            type_counts[obj_type] = type_counts.get(obj_type, 0) + 1
        print(f"   🏷️  Types d'objets: {dict(type_counts)}")
    
    # Informations sur la répartition temporelle
    frame_numbers = [int(f) for f in project['annotations'].keys()]
    if frame_numbers:
        print(f"\n📈 RÉPARTITION TEMPORELLE:")
        print(f"   📊 Première frame annotée: {min(frame_numbers)}")
        print(f"   📊 Dernière frame annotée: {max(frame_numbers)}")
        print(f"   📊 Plage totale: {max(frame_numbers) - min(frame_numbers) + 1} frames")
    
    print(f"\n✅ Propagation bidirectionnelle terminée avec succès!")
    print(f"💡 Le projet est maintenant prêt pour la sauvegarde et la visualisation")

🎯 MODE SEGMENTATION ACTIVÉ
🎯 CALCUL DES BORNES DE SEGMENTATION:
   📍 Frame de référence: 280
   📉 Offset avant: 50 frames
   📈 Offset après: 50 frames
   🎬 Segment original: frames 230 à 330
   🎬 Segment traité: indices 76 à 110
   📊 Nombre de frames à traiter: 35
🎬 EXTRACTION DU SEGMENT:
   📹 Vidéo: ..\data\videos\SD_13_06_2025_cam1_PdB_S1_T959s_1.mp4
   📁 Destination: ..\data\videos\outputs\SD_13_06_2025_cam1_PdB_S1_T959s_1\frames
   🎯 Segment: frames 230 à 330
   ⏯️  Intervalle: 3
🔄 34 frames existantes - SUPPRESSION et ré-extraction...
🗑️  Frames existantes supprimées
📊 Vidéo: 624 frames total, 25.0 FPS
📊 Nommage: séquentiel de 00000.jpg à 00033.jpg
📊 Progrès: 10 frames extraites (27.0%)
📊 Progrès: 20 frames extraites (57.0%)
📊 Progrès: 30 frames extraites (87.0%)
✅ 34 frames du segment extraites
📋 Nommage: 00000.jpg à 00033.jpg
✅ 34 frames du segment extraites
🎯 SEGMENT CONFIGURÉ:
   📊 Frames du segment: 101
   🎬 Plage: 230 à 330
   📍 Frame de référence: 280
🎯 Utilisation de la pr

propagate in video:  56%|█████▌    | 10/18 [02:00<02:26, 18.27s/it]

     📊 Frames traitées (reverse): 10


propagate in video: 100%|██████████| 18/18 [05:20<00:00, 17.79s/it]


   ✅ Phase reverse terminée: 18 frames

🔄 PHASE 2: Propagation avant (frame 17 → 33)


propagate in video: 100%|██████████| 17/17 [06:40<00:00, 23.57s/it]

   ✅ Phase forward terminée: 16 frames

✅ Propagation bidirectionnelle terminée:
   📊 Total frames traitées: 34
   🎯 Mode segmentation: Frame d'ancrage à l'index 17 dans le segment
   📍 Segment original: frames 230 à 330

📊 RÉSULTATS PROPAGATION BIDIRECTIONNELLE:
   🎯 Frame d'ancrage: 280 (original) → 17 (traitée)
   🎬 Frames originales: 624
   🎬 Frames traitées: 208
   📍 Frames avec annotations: 34
   📍 Annotations totales: 442
   🎯 Objets suivis: 13
   ⏯️  Intervalle frames: 3

🗂️  MAPPING FRAMES (échantillon):
   📋 Format: (frame_originale → frame_traitée)
   📋 Échantillon: [(230, 0), (233, 1), (236, 2), (239, 3), (242, 4), (245, 5), (248, 6), (251, 7), (254, 8), (257, 9)]
   📋 ... et 24 autres frames
   ⚠️  Attention: Frame d'ancrage 280 non trouvée dans le mapping
   🏷️  Types d'objets: {'player': 12, 'ball': 1}

📈 RÉPARTITION TEMPORELLE:
   📊 Première frame annotée: 0
   📊 Dernière frame annotée: 33
   📊 Plage totale: 34 frames

✅ Propagation bidirectionnelle terminée avec succès

In [13]:
# =============================================================================
# 💾 SAUVEGARDE DES RÉSULTATS
# =============================================================================

def save_project_results(cfg, project, verbose=True):
    """Sauvegarde les résultats du projet avec gestion d'erreurs"""
    
    if verbose:
        print(f"💾 Sauvegarde des résultats...")
        print(f"   📄 Fichier JSON: {cfg.output_json_path}")
    
    try:
        # Sauvegarde du JSON principal
        with open(cfg.output_json_path, 'w', encoding='utf-8') as f:
            json.dump(project, f, indent=2, ensure_ascii=False)
        
        if verbose:
            print(f"✅ Fichier JSON sauvé: {cfg.output_json_path}")
            
        # Sauvegarde d'une version compacte (sans indent pour économiser l'espace)
        compact_json_path = cfg.output_dir / f"{cfg.VIDEO_NAME}_project_compact.json"
        with open(compact_json_path, 'w', encoding='utf-8') as f:
            json.dump(project, f, separators=(',', ':'), ensure_ascii=False)
        
        if verbose:
            # Tailles des fichiers
            json_size = cfg.output_json_path.stat().st_size / 1024**2  # MB
            compact_size = compact_json_path.stat().st_size / 1024**2  # MB
            print(f"   📄 Version formatée: {json_size:.1f}MB")
            print(f"   📄 Version compacte: {compact_size:.1f}MB")
            
        return True
        
    except Exception as e:
        print(f"❌ Erreur lors de la sauvegarde: {e}")
        return False

def display_final_statistics(cfg, project):
    """Affiche les statistiques finales du projet"""
    
    # Calcul des statistiques
    total_annotations = sum(len(annotations) for annotations in project['annotations'].values())
    unique_frames = len(project['annotations'])
    
    # Calcul de la taille des données
    json_size = cfg.output_json_path.stat().st_size / 1024**2 if cfg.output_json_path.exists() else 0
    
    print(f"\n📊 RÉSULTATS FINAUX:")
    print(f"   🎬 Frames originales: {project['metadata']['frame_count_original']:,}")
    print(f"   🎬 Frames traitées: {project['metadata']['frame_count_processed']:,}")
    print(f"   📍 Frames avec annotations: {unique_frames:,}")
    print(f"   📍 Annotations totales: {total_annotations:,}")
    print(f"   🎯 Objets suivis: {len(project['objects'])}")
    print(f"   ⏯️  Intervalle frames: {project['metadata']['frame_interval']}")
    print(f"   📄 Taille JSON: {json_size:.1f}MB")
    print(f"   📁 Dossier de sortie: {cfg.output_dir}")
    
    # Affichage d'un échantillon du mapping
    sample_mapping = [(i, v) for i, v in enumerate(project['metadata']['frame_mapping'][:50]) if v is not None]
    print(f"   🗂️  Mapping échantillon (original→traité): {sample_mapping[:5]}...")
    
    # Résumé par type d'objet
    if project['objects']:
        type_counts = {}
        for obj_data in project['objects'].values():
            obj_type = obj_data.get('type', 'unknown')
            type_counts[obj_type] = type_counts.get(obj_type, 0) + 1
        print(f"   🏷️  Types d'objets: {dict(type_counts)}")
def create_colab_backup(cfg, use_timestamp=False, custom_suffix=""):
    """Crée une sauvegarde sur Google Drive pour Colab"""
    
    if not cfg.USING_COLAB:
        print("🖥️ Mode local: sauvegarde Google Drive skippée")
        return False
    
    try:
        from google.colab import drive
        import shutil
        from datetime import datetime
        
        print("🔬 Mode Colab: Création de la sauvegarde Google Drive...")
        
        # Monter le Drive
        print("   📁 Montage Google Drive...")
        drive.mount('/content/drive', force_remount=False)
        
        # Construction du nom selon les options
        if use_timestamp:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            zip_name = f"{cfg.VIDEO_NAME}_{timestamp}"
        elif custom_suffix:
            zip_name = f"{cfg.VIDEO_NAME}_{custom_suffix}"
        else:
            zip_name = cfg.VIDEO_NAME
        
        drive_zip_path = f'/content/drive/MyDrive/{zip_name}'
        
        # Vérifier si le fichier existe déjà
        existing_zip = Path(f"{drive_zip_path}.zip")
        if existing_zip.exists():
            existing_size = existing_zip.stat().st_size / 1024**2
            print(f"   🔄 Fichier existant trouvé: {zip_name}.zip ({existing_size:.1f}MB) - sera écrasé")
        
        print(f"   📦 Création du ZIP: {zip_name}.zip")
        
        # Créer le ZIP avec tout le dossier videos
        shutil.make_archive(drive_zip_path, 'zip', str(cfg.videos_dir))
        
        # Vérifier la taille
        zip_path = Path(f"{drive_zip_path}.zip")
        if zip_path.exists():
            zip_size = zip_path.stat().st_size / 1024**2  # MB
            print(f"✅ Sauvegarde créée: {zip_name}.zip ({zip_size:.1f}MB)")
            print(f"   📁 Emplacement: MyDrive/{zip_name}.zip")
            return True
        else:
            print("❌ Erreur: fichier ZIP non créé")
            return False
            
    except Exception as e:
        print(f"❌ Erreur sauvegarde Google Drive: {e}")
        return False

def cleanup_temporary_files(cfg):
    """Nettoie les fichiers temporaires (optionnel)"""
    
    print("🧹 Nettoyage optionnel...")
    
    # Comptage des frames
    frame_count = len(list(cfg.frames_dir.glob("*.jpg")))
    frame_size = sum(f.stat().st_size for f in cfg.frames_dir.glob("*.jpg")) / 1024**2
    
    print(f"   🖼️  Frames extraites: {frame_count} fichiers ({frame_size:.1f}MB)")
    print(f"   💡 Pour libérer l'espace, vous pouvez supprimer: {cfg.frames_dir}")
    print(f"   💡 Les frames peuvent être ré-extraites avec FORCE_EXTRACTION=True")

# =============================================================================
# 🚀 SAUVEGARDE AVEC ÉCRASEMENT PAR DÉFAUT
# =============================================================================

# Vérification des prérequis
if not hasattr(cfg, 'project') or cfg.project is None:
    print("❌ Projet non disponible. Exécutez d'abord la propagation.")
else:
    print(f"💾 Démarrage de la sauvegarde...")
    
    # Sauvegarde principale
    save_success = save_project_results(cfg, cfg.project)
    
    if save_success:
        # Affichage des statistiques
        display_final_statistics(cfg, cfg.project)
        
        # Sauvegarde Colab si applicable
        if cfg.USING_COLAB:
            print(f"\n🔬 Sauvegarde Google Drive...")
            
            # Mode par défaut : écrase l'ancienne version
            colab_success = create_colab_backup(cfg)  # use_timestamp=False par défaut
            
            # Si vous voulez un timestamp occasionnellement, décommentez :
            # colab_success = create_colab_backup(cfg, use_timestamp=True)
            
            if colab_success:
                print("✅ Sauvegarde Google Drive réussie!")
        
        # Nettoyage optionnel
        print(f"\n🧹 Informations de nettoyage:")
        cleanup_temporary_files(cfg)
        
        print(f"\n🎉 Pipeline SAM2 terminé avec succès!")
        print(f"📄 Fichier principal: {cfg.output_json_path}")
        if cfg.USING_COLAB:
            print(f"☁️ Sauvegarde Drive: MyDrive/{cfg.VIDEO_NAME}.zip")
            
    else:
        print("❌ Échec de la sauvegarde")

💾 Démarrage de la sauvegarde...
💾 Sauvegarde des résultats...
   📄 Fichier JSON: ..\data\videos\outputs\SD_13_06_2025_cam1_PdB_S1_T959s_1\SD_13_06_2025_cam1_PdB_S1_T959s_1_project.json
✅ Fichier JSON sauvé: ..\data\videos\outputs\SD_13_06_2025_cam1_PdB_S1_T959s_1\SD_13_06_2025_cam1_PdB_S1_T959s_1_project.json
   📄 Version formatée: 0.4MB
   📄 Version compacte: 0.2MB

📊 RÉSULTATS FINAUX:
   🎬 Frames originales: 624
   🎬 Frames traitées: 208
   📍 Frames avec annotations: 34
   📍 Annotations totales: 442
   🎯 Objets suivis: 13
   ⏯️  Intervalle frames: 3
   📄 Taille JSON: 0.4MB
   📁 Dossier de sortie: ..\data\videos\outputs\SD_13_06_2025_cam1_PdB_S1_T959s_1
   🗂️  Mapping échantillon (original→traité): []...
   🏷️  Types d'objets: {'player': 12, 'ball': 1}

🧹 Informations de nettoyage:
🧹 Nettoyage optionnel...
   🖼️  Frames extraites: 34 fichiers (24.8MB)
   💡 Pour libérer l'espace, vous pouvez supprimer: ..\data\videos\outputs\SD_13_06_2025_cam1_PdB_S1_T959s_1\frames
   💡 Les frames peuv